In [11]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split,KFold
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import Ridge, ElasticNet, Lasso, HuberRegressor
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor,AdaBoostRegressor,ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor
import xgboost as xgb
import lightgbm as lgb
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.pipeline import make_pipeline
from sklearn.kernel_ridge import KernelRidge
from bayes_opt import BayesianOptimization

In [12]:
#Input: Resultados de una comparacion de modelos
#Output: los resultados en forma de tabla
def get_models_table(models):
    models_table={}
    model_names=[get_model_name(model) for model in models]
    models_table['Regressor']=model_names
    models_table['model']=models
    models_table['error']=np.repeat(np.nan,len(models))
    models_table['params']=[{} for i in range(len(models))]

    return pd.DataFrame(models_table).set_index('Regressor')



def remove_outlier_predictions(y_pred,y_train):
    min_val, max_val = y_train.min(), y_train.max()
    result=y_pred.copy()
    result[y_pred<min_val]= min_val
    result[y_pred>max_val]= max_val
    
    return result


#Input: Un modelo
#Output: El nombre del modelo
def get_model_name(model):
    return str(model.__class__).split('.')[-1].split("'")[0]


#Input: Un modelo y un dataset separado en train y test
#Output: Obtiene la prediccion del modelo entrenado con el train sobre el conjunto de test
def fit_predict(model,X,y,X_test):
    model.fit(X,y)
    return remove_outlier_predictions(model.predict(X_test),y)


#Input: Un modelo y un dataset
#Output: Obtiene el error de cross validation del modelo sobre el dataset
def error_cv(model,X, y, verbose=0, metric='neg_mean_absolute_error'):
    return abs(cross_val_score(model, X, y, scoring = metric, n_jobs=-1, cv=8,verbose=verbose)).mean()


#Input: Un modelo, un dataset y un conjunto de parametros
#Output: Estima los parametros del modelo con cross validation y devuelve el error de la mejor cmbinacion (y la mejor combinacion)
def error_cv_param_grid(model,X, y,param_grid, verbose=0, metric='neg_mean_absolute_error'):
    grid_model=GridSearchCV(model,param_grid,scoring=metric,n_jobs=-1,cv=8,refit=False,verbose=verbose).fit(X,y)
    return abs(grid_model.best_score_), grid_model.best_params_


#Input: Un modelo, un dataset y (opcional) Un grid de parámetros
#Output: Devuelve una tabla con la comparativa de los modelos en terminos de error sobre el dataset (y sus parametros optimos si estimate_params=True)
def compare_models(models_table, X, y, estimate_params=False, verbose=0, metric='neg_mean_absolute_error'):
    
    errors=[]
    params=[]
    for i in range(models_table.shape[0]):
        
        model=models_table['model'].iloc[i] 
        if estimate_params:
            score,param = error_cv_param_grid(model,X, y,estimate_params[i],verbose=verbose, metric=metric)
            params.append(param)
        
        else:
            score=error_cv(models[i],X, y,verbose=verbose ,metric=metric)
        
        if verbose>0:
            print(models_table.index[i],': ',score)

        errors.append(score)
    
    models_table['error']=errors
    if estimate_params:
        models_table['params']=params
        
        
              
class Stacking_model(BaseEstimator, RegressorMixin):
    def __init__(self, base_models, meta_model, n_folds=5, metric=mean_absolute_error):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
        self.metric = metric
   
    def fit(self, X, y):
        
        X_matrix=X
        y_matrix=y
        if type(X)== pd.core.frame.DataFrame:
            X_matrix=X_matrix.as_matrix()
            
        if type(y)==pd.core.series.Series:    
            y_matrix=y_matrix.as_matrix()
        
        
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X_matrix, y_matrix):
                instance = clone(model)
                self.base_models_[i].append(instance)
                
                instance.fit(X_matrix[train_index], y_matrix[train_index])
                y_pred = instance.predict(X_matrix[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    def predict(self, X_test):
        df_test=pd.DataFrame()
        for i, base_models in enumerate(self.base_models_):
            df_test[i]=np.zeros(X_test.shape[0])
            for model in base_models:
                df_test[i]+=model.predict(X_test)
                
            df_test[i]/=len(base_models)
        
        return self.meta_model_.predict(df_test)
    
    
    def score(self,X_test,y_test):
        return self.metric(y_test,self.predict(X_test))

In [18]:
folder='Total'

traindata=pd.read_csv(folder+'/traindata.csv')#reading the data
testdata=pd.read_csv(folder+'/TEST.csv')#reading the data

traindata=traindata.drop('ID_Customer',axis=1)
test_ids=testdata['ID_Customer'] #Nos lo guardamos para submision
testdata=testdata.drop('ID_Customer',axis=1)

traindata=traindata[traindata['Poder_Adquisitivo']<1000000]

print('Number of rows and columns of the training set: ',traindata.shape)
print('Number of rows and columns of the test set: ',traindata.shape)

X_traindata=traindata.drop('Poder_Adquisitivo',axis=1)
y_traindata=traindata['Poder_Adquisitivo']

Number of rows and columns of the training set:  (363801, 104)
Number of rows and columns of the test set:  (363801, 104)


In [19]:
models=[
    Ridge(),
    Lasso(),
    ElasticNet(),
    KNeighborsRegressor(),
    HuberRegressor(),
    GradientBoostingRegressor(),
    AdaBoostRegressor(),
    RandomForestRegressor(),
    ExtraTreesRegressor(),
    xgb.XGBRegressor()   
]


models_table=get_models_table(models)

    
param_grid_list=[
    {'alpha':[0.05,0.1,0.3,0.6,1,1.5,3,5,10,15,30,50,80]},
    {'alpha':[0.0001, 0.0003, 0.0006, 0.001, 0.003, 0.006, 0.01, 0.03, 0.06, 0.1, 0.3, 0.6, 1.0]},
    {'l1_ratio':[0.1, 0.3, 0.5, 0.7, 0.9, 1],'alpha':[0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5]},
    {'n_neighbors':[3,5,7,9]},
    {'epsilon':[1.0,1.2,1.35,1.5,1.7,2.0], 'alpha':[0.00005,0.0001,0.0003,0.0006,0.0009,0.0012]},
    {'learning_rate': [0.1, 0.05, 0.01], 'max_depth': [4, 6, 8, 10, 12], 'n_estimators': range(50, 200, 25)},
    {'learning_rate': [0.5, 1, 3,5], 'n_estimators': range(50, 200, 25)},
    {'max_features':['auto', 'sqrt', 40], 'max_depth': [4, 6, 8, 10, None], 'n_estimators': range(50, 300, 50)},
    {'max_features':['auto', 'sqrt', 40], 'max_depth': [4, 6, 8, 10, None], 'n_estimators': range(50, 300, 50)},
    {}
]


#separamos nuestro conjunto de train en train y validacion
X_train, X_val, y_train, y_val = train_test_split(X_traindata, y_traindata, test_size=0.33)

In [ ]:
#Comparamos modelos (cada uno con su mejor combinacion de parametros)
compare_models(models_table, X_train, y_train, param_grid_list, verbose=4)

#Alternativamente podemos Comparar modelos cada uno con su combinacion de parametros por defecto (mas rapido)
#compare_models(models_table, X_traindata, y_traindata, verbose=4)
models_table

In [ ]:
#Escogemos un modelo y sus parametros en base a los resultados obtenidos arriba
#Validamos el modelo obteniendo el error para el conjunto de test
key=models_table['error'].argmin()

best_model=models_table.loc[key,'model'].set_params(**models_table.loc[key,'params'])
print('Validation mean absolute error: ',mean_absolute_error(y_val,fit_predict(best_model,X_train,y_train,X_val)))

In [ ]:
best_model.fit(X_traindata, y_traindata)

submision=pd.DataFrame()
submision['ID_Customer']=test_ids
submision['PA_Est']=fit_predict(best_model,X_traindata,y_traindata,testdata)
print('The description of the submision:\n',submision.describe())
submision.to_csv('Test_Mission.txt',index=False)

# De aquí para abajo sin pruebas, así que ni caso

In [20]:
y_pred6=fit_predict(Stacking_model([Ridge(),Ridge()],Ridge()),X_train,y_train,X_val)

In [21]:
mean_absolute_error(y_val,y_pred6)

5185.6150812617389

In [22]:
error_cv(Stacking_model([Ridge(),Ridge()],Ridge()),X_traindata,y_traindata,verbose=4)

[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV] ....................... , score=-5740.981967166104, total=  17.8s
[CV]  ................................................................
[CV] ....................... , score=-5323.258109960747, total=  18.4s
[CV]  ................................................................
[CV] ....................... , score=-5442.497596179486, total=  19.4s
[CV]  ................................................................
[CV] ....................... , score=-4942.859665523144, total=  21.1s
[CV]  ................................................................


[Parallel(n_jobs=-1)]: Done   4 out of   8 | elapsed:   24.0s remaining:   24.0s


[CV] ....................... , score=-4789.555751520681, total=  19.2s
[CV] ....................... , score=-5088.686880906654, total=  20.6s
[CV] ........................ , score=-5514.05504801813, total=  20.0s
[CV] ....................... , score=-5059.818126251833, total=  18.9s


[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:   43.0s finished


5237.7141431908476

In [ ]:
y_pred=fit_predict(GradientBoostingRegressor(n_estimators=3000,loss='huber'),X_train,y_train,X_val)

In [ ]:
y_pred2=fit_predict(xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1),X_train,y_train,X_val)

In [ ]:
y_pred3=fit_predict(lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11),X_train,y_train,X_val)

In [ ]:
mean_absolute_error(y_val,y_pred)

In [ ]:
mean_absolute_error(y_val,y_pred2)

In [ ]:
mean_absolute_error(y_val,y_pred3)

In [ ]:
mean_absolute_error(y_val,y_pred4)

In [ ]:
mae_cv(xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1),X_train,y_train)

In [23]:
params = {
        'silent': True,
        'eval_metric': 'mae',
        'verbose_eval': True
}

In [24]:
def xgb_evaluate(min_child_weight,
                 colsample_bytree,
                 max_depth,
                 subsample,
                 gamma,
                 alpha):

    params['min_child_weight'] = int(min_child_weight)
    params['cosample_bytree'] = max(min(colsample_bytree, 1), 0)
    params['max_depth'] = int(max_depth)
    params['subsample'] = max(min(subsample, 1), 0)
    params['gamma'] = max(gamma, 0)
    params['alpha'] = max(alpha, 0)

    return -mae_cv(xgb.XGBRegressor(**params),X_train,y_train)

In [25]:
xgbBO = BayesianOptimization(xgb_evaluate, {'n_estimators':(100,3000),
                                            'learning_rate':(0.05,0.5),
                                            'min_child_weight': (1, 20),
                                            'colsample_bytree': (0.1, 1),
                                            'max_depth': (5, 15),
                                            'subsample': (0.5, 1),
                                            'gamma': (0, 10),
                                            'alpha': (0, 10),
                                            })

xgbBO.maximize(init_points=5, n_iter=150)

SyntaxError: invalid syntax (<ipython-input-25-882cd3945410>, line 3)